In [37]:
!pip install langdetect

     |████████████████████████████████| 981 kB 5.9 MB/s eta 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=d755e201b825a2b91d5f18e285a402c9e996a669d3d52bb40ea4592fb2443986
  Stored in directory: /Users/casbertrams/Library/Caches/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [91]:
import duckdb
import pandas as pd
from functools import reduce
import numpy as np


In [2]:
conn = duckdb.connect('db/db.duckdb', read_only=False)
print(conn.execute('PRAGMA show_tables').fetchdf()['name'].tolist())


['end_year', 'labels', 'num_votes', 'original_title', 'primary_title', 'runtime', 'start_year']


In [7]:
print(conn.execute('SELECT * FROM original_title').fetchdf())

         tconst                  original_title
0     tt0010600                       Die Puppe
1     tt0011841                   Way Down East
2     tt0012494                    Der mude Tod
3     tt0015163                   The Navigator
4     tt0016220        The Phantom of the Opera
...         ...                             ...
7954  tt9625664                                
7955  tt9741310                           Slaxx
7956  tt9742392                         Kindred
7957  tt9850386                                
7958  tt9911196  De beentjes van Sint-Hildegard

[7959 rows x 2 columns]


In [92]:
tables = conn.execute('PRAGMA show_tables').fetchdf()['name'].tolist()
frames = []
for i in tables:
    frames.append(conn.execute(f"SELECT * FROM {i}").fetchdf())
merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['tconst'], how='outer'), frames)
print(f"Any NaN values in the df: {merged_df.isnull().values.any()}")

labels = merged_df['labels']
labels = np.array(labels.astype('int').tolist())
merged_df = merged_df.drop('labels', 1)

Any NaN values in the df: False


Add value to indicate wether the title has been changed

In [93]:
renamed = []
for i in range(len(merged_df)):
    curr_original = merged_df.iloc[i]['original_title']
    curr_primary = merged_df.iloc[i]['primary_title']
    if curr_original != "" and curr_primary != curr_original:
        renamed.append(1)
    else:
        renamed.append(0)
    
merged_df['renamed'] = renamed

Transform dataframe to array

In [94]:
print(merged_df.dtypes)
merged_df = merged_df.drop('original_title', 1)
merged_df = merged_df.drop('primary_title', 1)
merged_df = merged_df.drop('tconst', 1)
full_array = merged_df.to_numpy()

tconst             object
end_year            int32
num_votes           int32
original_title     object
primary_title      object
runtime_minutes     int32
start_year          int32
renamed             int64
dtype: object


In [95]:
print(full_array)

[[   0 1898   66 1919    1]
 [   0 5376  145 1920    0]
 [   0 5842   97 1921    1]
 ...
 [   0 1719  101 2020    0]
 [   0 4144  111 2020    0]
 [   0 3242  103 2020    1]]


Train / test split

In [102]:
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()
full_array = standardizer.fit_transform(full_array)

X_train = full_array[:6300]
y_train = labels[:6300]

X_test = full_array[6300:]
y_test = labels[6300:]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)



(6300, 5)
(6300,)
(1659, 5)
(1659,)


### Logistic Regression

In [104]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

LogisticRegression()

In [106]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)

TN, FP, FN, TP = confusion_matrix(y_test, predictions).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))

True Positive(TP)  =  465
False Positive(FP) =  200
True Negative(TN)  =  696
False Negative(FN) =  298
Accuracy of the binary classification = 0.700


### Keras

In [108]:
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense        # importing Dense layers
import keras.optimizers
import tensorflow as tf

In [113]:
basic_model = Sequential()
# Adding layers to the model
# First layers: 16 neurons/perceptrons that takes the input and uses 'sigmoid' activation function.
basic_model.add(Dense(units = 16 , activation = 'sigmoid', input_shape = (5,))) 
# Second layer: 1 neuron/perceptron that takes the input from the 1st layers and gives output as 0 or 1.Activation used is 'Hard Sigmoid'
basic_model.add(Dense(1, activation = 'hard_sigmoid'))




In [114]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.5, momentum=0.9, nesterov=True)
basic_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [116]:
basic_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
basic_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
197/197 [==============================] - 0s 741us/step - loss: 0.6435 - accuracy: 0.6513
Epoch 2/100
197/197 [==============================] - 0s 694us/step - loss: 0.6427 - accuracy: 0.6554
Epoch 3/100
197/197 [==============================] - 0s 766us/step - loss: 0.6422 - accuracy: 0.6594
Epoch 4/100
197/197 [==============================] - 0s 696us/step - loss: 0.6418 - accuracy: 0.6625
Epoch 5/100
197/197 [==============================] - 0s 760us/step - loss: 0.6415 - accuracy: 0.6627
Epoch 6/100
197/197 [==============================] - 0s 738us/step - loss: 0.6413 - accuracy: 0.6630
Epoch 7/100
197/197 [==============================] - 0s 871us/step - loss: 0.6411 - accuracy: 0.6652
Epoch 8/100
197/197 [==============================] - 0s 896us/step - loss: 0.6410 - accuracy: 0.6662
Epoch 9/100
197/197 [==============================] - 0s 762us/step - loss: 0.6409 - accuracy: 0.6665
Epoch 10/100
197/197 [==============================] - 0s 717us/step - l

197/197 [==============================] - 0s 744us/step - loss: 0.6396 - accuracy: 0.6719
Epoch 80/100
197/197 [==============================] - 0s 731us/step - loss: 0.6396 - accuracy: 0.6719
Epoch 81/100
197/197 [==============================] - 0s 757us/step - loss: 0.6396 - accuracy: 0.6725
Epoch 82/100
197/197 [==============================] - 0s 675us/step - loss: 0.6396 - accuracy: 0.6722
Epoch 83/100
197/197 [==============================] - 0s 645us/step - loss: 0.6396 - accuracy: 0.6722
Epoch 84/100
197/197 [==============================] - 0s 676us/step - loss: 0.6396 - accuracy: 0.6737
Epoch 85/100
197/197 [==============================] - 0s 762us/step - loss: 0.6396 - accuracy: 0.6719
Epoch 86/100
197/197 [==============================] - 0s 720us/step - loss: 0.6396 - accuracy: 0.6722
Epoch 87/100
197/197 [==============================] - 0s 649us/step - loss: 0.6396 - accuracy: 0.6719
Epoch 88/100
197/197 [==============================] - 0s 693us/step - loss:

In [117]:
# Test, Loss and accuracy
loss_and_metrics = basic_model.evaluate(X_test, y_test)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

52/52 [==============================] - 0s 521us/step - loss: 0.6618 - accuracy: 0.6757
Loss =  0.6617915630340576
Accuracy =  0.6757082343101501


### Keras 2

In [119]:
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense        # importing Dense layers
import keras.optimizers
import tensorflow as tf

In [124]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(5,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [125]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=1)

Epoch 1/10
6300/6300 [==============================] - 4s 630us/step - loss: 0.6069 - accuracy: 0.6749
Epoch 2/10
6300/6300 [==============================] - 4s 631us/step - loss: 0.5869 - accuracy: 0.6935
Epoch 3/10
6300/6300 [==============================] - 4s 631us/step - loss: 0.5746 - accuracy: 0.7022
Epoch 4/10
6300/6300 [==============================] - 4s 634us/step - loss: 0.5677 - accuracy: 0.7151
Epoch 5/10
6300/6300 [==============================] - 4s 648us/step - loss: 0.5611 - accuracy: 0.7202
Epoch 6/10
6300/6300 [==============================] - 4s 643us/step - loss: 0.5568 - accuracy: 0.7156
Epoch 7/10
6300/6300 [==============================] - 4s 672us/step - loss: 0.5540 - accuracy: 0.7241
Epoch 8/10
6300/6300 [==============================] - 5s 724us/step - loss: 0.5498 - accuracy: 0.7252
Epoch 9/10
6300/6300 [==============================] - 4s 657us/step - loss: 0.5476 - accuracy: 0.7270
Epoch 10/10
6300/6300 [==============================] - 4s 671u

In [131]:
# Test, Loss and accuracy
loss_and_metrics = model.evaluate(X_test, y_test)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

52/52 [==============================] - 0s 610us/step - loss: 0.5748 - accuracy: 0.7058
Loss =  0.5748496055603027
Accuracy =  0.705846905708313


In [133]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);

In [134]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Test, Loss and accuracy
print(predictions)



[1 0 0 ... 0 0 0]
